In [57]:
import requests

def search_searxng(query, instance_url="http://localhost:32768"):
    response = requests.post(
        f"{instance_url}/search",
        data={
            "q": query,
            "categories": "general",
            "language": "en",
            "format": "json"
        },
        timeout=10
    ) 
    
    if response.ok:
        results = response.json().get("results", [])
        return [(r["title"], r["url"]) for r in results]
    else:
        raise Exception(f"Search failed: {response.status_code} - {response.text}")

# Example usage
results = search_searxng("A good phone repair business")
for title, url in results:
    print(f"{title}: {url}")

r/sweatystartup on Reddit: Is Starting A Phone Repair Business Worth It?: https://www.reddit.com/r/sweatystartup/comments/ph4a8o/is_starting_a_phone_repair_business_worth_it/
The Ultimate Guide To Starting A Phone Repair Business - RemOnline: https://remonline.app/blog/how-to-start-a-phone-repair-business/
How to Start a 7-Figure Cell Phone Repair Business (2025) - UpFlip: https://www.upflip.com/blog/cell-phone-repair-business
The Ultimate Guide To Starting A Successful Phone Repair Business: https://orderry.com/blog/how-to-start-a-phone-repair-business/
10 Tips To Grow Your Cell Phone Repair Business - Mighty Sites: https://mightysites.com/articles/10-tips-to-grow-your-cell-phone-repair-business
How To Grow a Phone Repair Business - HostPapa: https://www.hostpapa.com/ideas/business/how-to-grow-a-phone-repair-business/
Is a cell phone repair shop a good business to own? - Quora: https://www.quora.com/Is-a-cell-phone-repair-shop-a-good-business-to-own
Cell Phone Repair Business Startup 

In [1]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import gradio as gr
import torch

import requests
from bs4 import BeautifulSoup

import pandas as pd
from tabulate import tabulate
import textwrap

/home/joshua/miniconda3/envs/llm-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/mnt/models/llm_storage/Llama-3.2-3B-Instruct"
#model_path = "/home/joshua/llms/llama3.1_PC_Build/Llama-3.1-8B-Instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    device_map="cuda:0"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.37it/s]


In [3]:
def format_messages(messages):
    prompt = ""
    for msg in messages:
        role = msg["role"]
        if role == "system":
            prompt += f"<|system|>\n{msg['content']}\n"
        elif role == "user":
            prompt += f"<|user|>\n{msg['content']}\n"
        elif role == "assistant":
            prompt += f"<|assistant|>\n{msg['content']}\n"
    prompt += "<|assistant|>\n"
    return prompt

def generate_response(messages, max_new_tokens):
    prompt = format_messages(messages)
    #print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        #top_p=0.95,
        #repetition_penalty=1.1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("<|assistant|>")[-1]

In [ ]:

# JOSHUA - NEEDS WORK, VERY SHIT
def generate_search_tag(natural_query: str) -> str:
    print(f"[1] Generating search tag for input: {natural_query}")

    messages = [
        {
            "role": "system",
            "content": (
                "You are an assistant that rewrites user queries into short, search-engine-friendly tags.\n"
                "Your goal is to identify the **type of business or service** the user is looking for.\n\n"
                "Rules:\n"
                "- Use plain nouns like 'pc store', 'vegan café', 'soup restaurant', etc.\n"
                "- No punctuation, no quotes.\n"
                "- No vague or low-information tags like 'pc's' or 'help'.\n"
            )
        },
        {
            "role": "user",
            "content": "Query: I want someone to fix my laptop\nSearch tag:"
        },
        {
            "role": "assistant",
            "content": "laptop repair service"
        },
        {
            "role": "user",
            "content": "Query: I want to buy a computer\nSearch tag:"
        },
        {
            "role": "assistant",
            "content": "computer retail store"
        },
        {
            "role": "user",
            "content": "Query: Looking fro some good soup\nSearch tag:"
        },
        {
            "role": "assistant",
            "content": "soup restaurant"
        },
        {
            "role": "user",
            "content": f"Query: {natural_query}\nSearch tag:"
        }
    ]

    response = generate_response(messages, max_new_tokens=30)
    tag = response.strip().split('\n')[0].strip()
    print(f"[1.2] Generated tag: {tag}")
    #tag = response.strip().split('Tag:')[-1].strip()
    return tag


# LOOK AT PARSING PURE HTML AS TEXT RATHER THAN JSON
def getCleanedText(html_text: str) -> str:
    print("[2] Cleaning HTML...")
    soup = BeautifulSoup(html_text, "html.parser")
    for tag in soup(["script", "style", "noscript", "footer", "header"]):
        tag.extract()
    text = soup.get_text(separator=" ", strip=True)
    cleaned = " ".join(text.split())
    print(f"[2.1] Cleaned text length: {len(cleaned)} chars")
    return cleaned

# COMBINE THIS WITH THE FUNCTION ABOVE OR WORK INTO MAIN LOOP
def getBusinessUrls(urls: list[str], searx_instance: str) -> list[dict]:
    print(f"[3] Fetching and cleaning {len(urls)} URLs")
    cleaned_pages = []
    for idx, url in enumerate(urls):
        print(f"[3.{idx}] Downloading: {url}")
        try:
            res = requests.post(
                f"{searx_instance}/search",
                data={
                    "q": url,
                    "categories": "general",
                    "language": "en",
                    "safesearch": "0",
                    "format": "json",
                    "engines": "bing,startpage,brave,mojeek,qwant"
                },
                timeout=10
            ) 
            res.raise_for_status()
            clean_text = getCleanedText(res.text)
            print(f"[3.{idx}.1] Page cleaned successfully ({len(clean_text)} chars)")
            cleaned_pages.append({"url": url, "text": clean_text})
        except Exception as e:
            print(f"[!] Failed to fetch {url}: {e}")
    return cleaned_pages

def relevanceCheck(cleaned_text: str, user_input: str) -> bool:
    print(f"[4] Relevance check...")
    limited_content = cleaned_text[:1000]

    messages = [
        {
            "role": "system",
            "content": (
                "You are a strict validator. Only reply with 'yes' or 'no'.\n\n"
                "Your task is to determine if the following page content is related to the user’s intent "
                "and describes a real-world business or organization that could be contacted.\n\n"
                "Guidelines:\n"
                "- Look for signs of actual businesses: names, product listings, services, contact info, locations.\n"
                "- Only say 'yes' to pages that clearly describe physical businesses or service providers.\n"
                "- Do NOT say 'yes' to pages like forums, blogs, Wikipedia, or generic search results.\n"
                "- NEVER say 'yes' if the content appears to be:\n"
                "    • JSON metadata\n"
                "    • CAPTCHA pages\n"
                "    • Access denied / blocked pages\n"
                "    • Search engine error messages or redirection notices\n"
            )
        },
        {
            "role": "user",
            "content": (
                f"Intent: {user_input}\n"
                f"Page content:\n{limited_content}\n"
                "yes or no:"
            )
        }
    ]

    response = generate_response(messages, max_new_tokens=2)
    decoded = response.lower().strip()
    print(f"[4.1] Relevance result: {decoded}")
    return decoded.startswith("yes")

def summarizeRelatedBusiness(page_text: str, user_input: str) -> str:
    print(f"[5] Summarizing page...")
    print(f"[5.1] Page Text: {page_text}")

    messages = [
        {"role": "system", "content": (
            "You are a business summarization assistant. Your job is to extract useful business information from webpages.\n"
            "Focus on identifying the **Name of business** **type of business**, **services they provide**, **industries or customers they serve**, and **location** or **email** if mentioned.\n"
            "Be concise but informative. This summary will be passed into another agent that writes cold outreach emails, so provide enough context for that.\n"
            "Avoid vague language. Only summarize if you detect an actual business presence (e.g., a service provider, store, firm, platform, etc)."
        )},
        {"role": "user", "content": (
            f"User is searching for: {user_input}\n"
            f"Here is the page content:\n"
            f"{page_text[:2000]}\n\n"
            "Summarize the business based on this content:"
        )}
    ]

    summary = generate_response(messages, max_new_tokens=400).strip()
    print(f"[5.2] Summary length: {len(summary)} chars")
    return summary


In [66]:
def search_and_summarize(business_query: str, top_k: int, pages: int, clean_limit: int):
    print(f"[0] Starting search and summarize for query: {business_query}")
    search_term = generate_search_tag(business_query)
    print(f"TEST: {search_term}")
    print(f"[0.1] Using search term: {search_term}")

    searx_instance = "http://localhost:32768"
    all_results = []

    # Collect multiple pages
    # JOSHUA - FIND A MORE DYNAMIC WAY TO DO THIS RATHER THAN ASSUMING 10 PAGES
    for page_num in range(pages):
        offset = page_num * 10  # Default engine page size
        params = {
            "q": search_term,
            "categories": "general",
            "language": "en",
            "safesearch": "0",
            "format": "json",
            "engines": "bing,startpage,brave,mojeek,qwant",
            "start": offset
        }

        try:
            response = requests.get(f"{searx_instance}/search", params=params, timeout=10)
            response.raise_for_status()
            page_results = response.json().get("results", [])
            print(f"[0.{page_num}] Retrieved {len(page_results)} results from page {page_num + 1}")
            all_results.extend(page_results)
        except Exception as e:
            print(f"[!] Error retrieving page {page_num + 1}: {e}")

    # Deduplicate and extract URLs
    urls = list({r["url"] for r in all_results if "url" in r})
    print(f"[0.2] Total unique URLs collected: {len(urls)}")

    cleaned_urls = urls if clean_limit is None else urls[:clean_limit]
    cleaned_pages = getBusinessUrls(cleaned_urls, searx_instance)
    summaries = []

    for i, page in enumerate(cleaned_pages):
        print(f"[6.{i}] Checking page: {page['url']}")
        if relevanceCheck(page["text"], business_query):
            summary = summarizeRelatedBusiness(page["text"], business_query)
            summaries.append((page["url"], summary))
            print(f"[+] Relevant page added: {page['url']}")
        else:
            print(f"[-] Page deemed not relevant: {page['url']}")
        if len(summaries) >= top_k:
            print(f"[✓] Reached top_k = {top_k} summaries")
            break

    print(f"\n\n[✓] Done. Collected {len(summaries)} relevant summaries.")
    return summaries




def display_summaries(summary_results, wrap_width=80):
    df = pd.DataFrame(summary_results, columns=["URL", "Summary"])
    wrapper = textwrap.TextWrapper(width=wrap_width)
    df["Summary"] = df["Summary"].apply(lambda text: "\n".join(wrapper.wrap(text)))

    # Print nicely using tabulate
    print(tabulate(df, headers="keys", tablefmt="fancy_grid", showindex=False))

#business = "I want a business that sells pc's"
business = "Looking for a furtnature store that sells office stuff"

summary_results = search_and_summarize(business, 5, 5, None)

display_summaries(summary_results, wrap_width=100)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[0] Starting search and summarize for query: Looking for a furtnature store that sells office stuff
[1] Generating search tag for input: Looking for a furtnature store that sells office stuff
[1.2] Generated tag: appliance repair
TEST: appliance repair
[0.1] Using search term: appliance repair
[0.0] Retrieved 30 results from page 1
[0.1] Retrieved 30 results from page 2
[0.2] Retrieved 30 results from page 3
[0.3] Retrieved 30 results from page 4
[0.4] Retrieved 30 results from page 5
[0.2] Total unique URLs collected: 35
[3] Fetching and cleaning 35 URLs
[3.0] Downloading: https://nextdoor.com/pages/appliance-works-west-valley-appliance-repair/
[2] Cleaning HTML...
[2.1] Cleaned text length: 6655 chars
[3.0.1] Page cleaned successfully (6655 chars)
[3.1] Downloading: https://www.appliancecareandrepair.com/sun-city-appliance-repair
[2] Cleaning HTML...
[2.1] Cleaned text length: 6021 chars
[3.1.1] Page cleaned successfully (6021 chars)
[3.2] Downloading: https://www.yelp.com/search?cfl

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[2] Cleaning HTML...
[2.1] Cleaned text length: 5970 chars
[3.34.1] Page cleaned successfully (5970 chars)
[6.0] Checking page: https://nextdoor.com/pages/appliance-works-west-valley-appliance-repair/
[4] Relevance check...
[4.1] Relevance result: no.
[-] Page deemed not relevant: https://nextdoor.com/pages/appliance-works-west-valley-appliance-repair/
[6.1] Checking page: https://www.appliancecareandrepair.com/sun-city-appliance-repair
[4] Relevance check...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

[4.1] Relevance result: no
[-] Page deemed not relevant: https://www.yelp.com/search?find_desc=appliance+repair+service&find_loc=Sun+City%2C+AZ+85351&msockid=0469f142fd876a5d0fb9e4b1fc846b8e
[6.30] Checking page: https://fallonsolutions.com.au/appliance-repairs/
[4] Relevance check...
[4.1] Relevance result: no
[-] Page deemed not relevant: https://fallonsolutions.com.au/appliance-repairs/
[6.31] Checking page: https://puls.com/local/az/sun-city/appliances
[4] Relevance check...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[4.1] Relevance result: no
[-] Page deemed not relevant: https://puls.com/local/az/sun-city/appliances
[6.32] Checking page: https://dynamicappliancerepairaz.com/sun-city-appliance-repair/
[4] Relevance check...
[4.1] Relevance result: no.
[-] Page deemed not relevant: https://dynamicappliancerepairaz.com/sun-city-appliance-repair/
[6.33] Checking page: https://hillsideappliancerepair.com/areas/sun-city-appliance-repair-services/
[4] Relevance check...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[4.1] Relevance result: no
[-] Page deemed not relevant: https://hillsideappliancerepair.com/areas/sun-city-appliance-repair-services/
[6.34] Checking page: https://www.glendaleapplianceco.com/sun-city-az/
[4] Relevance check...
[4.1] Relevance result: no
[-] Page deemed not relevant: https://www.glendaleapplianceco.com/sun-city-az/


[✓] Done. Collected 2 relevant summaries.
╒═════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ URL                                                     │ Summary                                                                                              │
╞═════════════════════════════════════════════════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ https://applianceworksaz.com/sun-city-appliance-repair/ │ The business is Appliance Works, and they are an appliance repair servic